# Deep Learning Modeling for SIMOC

Deep learning is a way of modeling a system by creating a set of tasks that a computer follows to learn about the system. Given the large amounts of data necessary to make these models work, deep learning has become extremely popular in the past few years and has been very successful at breaking down complex systems. 

Deep learning is ideal for SIMOC as it allows for a model to be built upon real data and could help us learn more about how a self-reliant colony may respond when given new tasks. To demonstrate how deep learning works, data from a pre-built model will be used to train and test this new model. The input will be oxygen and carbon dioxide levels and based on this input the model will try to predict how many people and square meters of plants were in that room at the time that that dataset was generated.

To start, we'll load in the necessary libraries. TensorFlow is the deep learning library that will be used to do the bulk of the model building and Numpy will be used for some minor array manipulation.

In [1]:
import tensorflow as tf
import numpy as np

Next, the data needs to be loaded in and separated into training and testing datasets. For simplicity, I have created training and testing datasets in CSV files in advance.

In [2]:
trainData = np.genfromtxt('trainData.csv', delimiter=',')
testData = np.genfromtxt('testData.csv', delimiter=',')

From there, the testing and training data will be separated into inputs and outputs that will be trained and tested against once the model is built

In [3]:
train_x = trainData[0:,0:23]
train_y = trainData[0:,[24,25]]
test_x = testData[0:,0:23]
test_y = testData[0:,[24,25]]

Now that the data is loaded in, we can start defining the model. Solving this problem could likely only require linear regression, so using a deep learning model is a little overkill and it will probably lead to some overfitting. However, since the final model will be using deep learning, using a deep learning problem for this data will most likely be accurate enough.

For this model, two hidden layers will be used (so it is technically a deep learning model) with 4 nodes each. The final solution will have 2 outputs, the number of plants and number of people, and the batch size will be set at 100 sets of data.

In [4]:
nodes_hl1 = 4
nodes_hl2 = 4
n_classes = 2
batch_size = 100

To define the input and output of this model, two tensorflow placeholder variables will be created.

In [5]:
x = tf.placeholder('float', [None, 23])
y = tf.placeholder('float')

Now we can continue to define the model. As mentioned before, this model will have two hidden layers each initialized with their own weights and biases set randomly at when created. These weights and biases will be adjusted over the course the of model's lifetime. The model then multiplies the given data by the weights and adds the bias to generate a projected value.

In [6]:
def neural_net(data):
    hidden_layer_1 = {'weights': tf.Variable(tf.random_normal([23,nodes_hl1],seed=1)),
                        'bias': tf.Variable(tf.random_normal([nodes_hl1],seed=1))}
    hidden_layer_2 = {'weights': tf.Variable(tf.random_normal([nodes_hl1,nodes_hl2],seed=1)),
                        'bias': tf.Variable(tf.random_normal([nodes_hl2],seed=1))}
    output_layer = {'weights': tf.Variable(tf.random_normal([nodes_hl2,n_classes],seed=1)),
                        'bias': tf.Variable(tf.random_normal([n_classes],seed=1))}

    # (data * weights) + bias

    l1 = tf.add(tf.matmul(data, hidden_layer_1['weights']), hidden_layer_1['bias'])
    l1 = tf.nn.relu(l1)
    l2 = tf.add(tf.matmul(l1, hidden_layer_2['weights']), hidden_layer_2['bias'])
    l2 = tf.nn.relu(l2)
    # Activation Function
    output = tf.add(tf.matmul(l2, output_layer['weights']), output_layer['bias'])
    return output

Finally, we can start training the model. To do this, a cost function is created based on the predictions and expected values. This cost function is then adjusted by tensorflow and weights are adjusted to produce a close estimate to our expected output. This cycle happens for each batch of data pulled from the dataset to find accurate values for the weights. Once the model has been trained, it is tested against new data find its accuracy

In [7]:
def train_NN(x):

    # Setup Model
    prediction = neural_net(x)

    #Produce Cost Function
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))

    # Optimizer
    optimize = tf.train.AdamOptimizer().minimize(cost)

    # Number of Back Prop Cycles
    cycles = 10

    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())
        for cycle in range(cycles):
            cycle_loss = 0
            i = 0
            while i < len(train_x):
                start = i
                end = batch_size+i
                batch_x = np.array(train_x[start:end])
                batch_y = np.array(train_y[start:end])
                _, c = sess.run([optimize,cost], feed_dict={x: batch_x, y: batch_y})
                cycle_loss += c
                i += batch_size

            print ("Epoch", cycle+1, "completed out of" , cycles)

        correct = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct,'float'))
        print ('Accuracy', accuracy.eval({x:test_x, y:test_y}))

Now, we can run the model and test it's accuracy.

In [8]:
train_NN(x)

('Epoch', 1, 'completed out of', 10)
('Epoch', 2, 'completed out of', 10)
('Epoch', 3, 'completed out of', 10)
('Epoch', 4, 'completed out of', 10)
('Epoch', 5, 'completed out of', 10)
('Epoch', 6, 'completed out of', 10)
('Epoch', 7, 'completed out of', 10)
('Epoch', 8, 'completed out of', 10)
('Epoch', 9, 'completed out of', 10)
('Epoch', 10, 'completed out of', 10)
('Accuracy', 0.98000002)


As expected, the model performed well against the tested data, however, it is slightly off due to likely overfitting. This is still a good example of how deep learning models can be used to predict values given a set of data and, if used in the final model, will help to predict patterns that may have been difficult to model by hand.